<a href="https://colab.research.google.com/github/thaonguyyen/project_chd/blob/main/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Read in cleaned training and testing data

In [16]:
train_df = pd.read_csv("https://raw.githubusercontent.com/thaonguyyen/project_chd/main/cleaned_train_data.csv")
test_df = pd.read_csv("https://raw.githubusercontent.com/thaonguyyen/project_chd/main/cleaned_test_data.csv")

train_df.describe()

,Unnamed: 0.1,Unnamed: 0,sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000
mean,1587.049001,2129.629916,0.438104,49.598324,1.905867,0.486460,8.960993,0.029014,0.006125,0.321728,0.026112,236.603804,132.751612,83.136041,25.883978,75.894907,82.013862,0.151838
std,918.982372,1233.694926,0.496234,8.537368,1.046872,0.499897,12.006022,0.167871,0.078035,0.467215,0.159495,44.850722,22.340205,11.943124,4.130808,12.044684,22.440695,0.358921
min,0.000000,1.000000,0.000000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000,83.500000,50.000000,15.540000,44.000000,40.000000,0.000000
25%,791.250000,1059.250000,0.000000,42.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,205.000000,117.000000,75.000000,23.092500,68.000000,72.000000,0.000000
50%,1588.500000,2139.500000,0.000000,49.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,233.000000,129.000000,82.000000,25.490000,75.000000,79.000000,0.000000
75%,2383.750000,3214.750000,1.000000,56.000000,3.000000,1.000000,20.000000,0.000000,0.000000,1.000000,0.000000,262.000000,144.000000,90.000000,28.130000,83.000000,85.000000,0.000000
max,3179.000000,4240.000000,1.000000,70.000000,4.000000,1.000000,70.000000,1.000000,1.000000,1.000000,1.000000,696.000000,295.000000,142.500000,56.800000,140.000000,394.000000,1.000000


Using k-nearest-neighbor model




In [17]:
from sklearn.neighbors import KNeighborsClassifier

X_train = train_df.drop("TenYearCHD", axis=1)
X_test = test_df.drop("TenYearCHD", axis=1)
y_train = train_df["TenYearCHD"]
y_test = test_df["TenYearCHD"]

N_train = len(y_train)
N_test = len(y_test)

k_test = np.arange(1,150)
Acc = np.zeros(150)

for k in range(150):
    fitted_model = KNeighborsClassifier(n_neighbors=k+1).fit(X_train.values, y_train)
    y_hat = fitted_model.predict(X_test.values) # Predict values for test set
    Acc[k] = np.sum( y_hat == y_test )/N_test # Accuracy on testing data

# Acc_max = np.max(Acc) # Find highest recorded Accuracy
# max_index = np.where(Acc==Acc_max) # Find the indices that equal the maximum
# k_star = max_index[0]+1 # Find the optimal value of k; why index+1?
# print(k_star)
Acc_max = np.max(Acc)
print("The highest recorded accuracy is: ", Acc_max)
max_index = np.where(Acc == Acc_max)[0]
k_star = max_index[0] + 1
print("The optimal k-value is:", k_star)

The highest recorded accuracy is:  0.8548076923076923
The optimal k-value is: 6


In [18]:
model = KNeighborsClassifier(n_neighbors=6).fit(X_train.values, y_train)
y_hat = model.predict(X_test.values)

In [19]:
# from sklearn.metrics import r2_score
# r2 = r2_score(y_test, y_hat)
# print(f"R^2 Score: {r2}")

# from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(y_test, y_hat)
# print("Accuracy:", accuracy)

accuracy = np.sum( y_hat == y_test )/N_test
print("The highest recorded accuracy is: ", accuracy)

The highest recorded accuracy is:  0.8548076923076923


In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy = accuracy_score(y_test, y_hat)
conf_matrix = confusion_matrix(y_test, y_hat)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")


Accuracy: 0.8548076923076923
Confusion Matrix:
[[889   0]
 [151   0]]


In [21]:
def maxmin(x):
    u = (x-min(x))/(max(x)-min(x))
    return u
X_maxmin_train = train_df.apply(maxmin)
X_maxmin_test = test_df.apply(maxmin)

In [22]:
def performance(y_test,y_hat): # Sample code to compute classifier performance
    tab = pd.crosstab(np.array(y_hat),np.array(y_test)) # Compute confusion matrix
    #
    # Get entries from the confusion matrix:
    tp = tab.iloc[0,0]
    tn = tab.iloc[1,1]
    fp = tab.iloc[0,1]
    fn = tab.iloc[1,0]
    tot = tp+tn+fp+fn
    #
    # Compute performance statistics:
    acc = (tp+tn)/tot
    spec = tn/(tp+fn)
    sens = tp/(tp+fn)
    mcc = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    #
    # Return a dictionary including the confusion matrix and metrics:
    return({'confusion_matrix':tab,
            'accuracy':acc,
            'specificity':spec,
            'sensitivity':sens,
            'mcc':mcc})

print(performance(y_test, y_hat))

IndexError: index 1 is out of bounds for axis 0 with size 1

Normalized

In [10]:
from sklearn.neighbors import KNeighborsClassifier

X_train = X_maxmin_train.drop("TenYearCHD", axis=1)
X_test = X_maxmin_test.drop("TenYearCHD", axis=1)
y_train = train_df["TenYearCHD"]
y_test = test_df["TenYearCHD"]

N_train = len(y_train)
N_test = len(y_test)

k_test = np.arange(1,150)
Acc = np.zeros(150)

for k in range(150):
    fitted_model = KNeighborsClassifier(n_neighbors=k+1).fit(X_train.values, y_train)
    y_hat = fitted_model.predict(X_test.values)
    Acc[k] = np.sum( y_hat == y_test )/N_test

Acc_max = np.max(Acc)
max_index = np.where(Acc == Acc_max)[0]
k_star = max_index[0] + 1
print("The optimal k-value is:", k_star)

The optimal k-value is: 11


In [11]:
model = KNeighborsClassifier(n_neighbors=11).fit(X_train, y_train)
y_hat = model.predict(X_test)
accuracy = np.sum( y_hat == y_test )/N_test
print("The highest recorded accuracy is: ", accuracy)

The highest recorded accuracy is:  0.8548076923076923


In [12]:
accuracy = accuracy_score(y_test, y_hat)
conf_matrix = confusion_matrix(y_test, y_hat)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")


Accuracy: 0.8548076923076923
Confusion Matrix:
[[878  11]
 [140  11]]


In [13]:
def performance(y_test,y_hat): # Sample code to compute classifier performance
    tab = pd.crosstab(np.array(y_hat),np.array(y_test)) # Compute confusion matrix
    #
    # Get entries from the confusion matrix:
    tp = tab.iloc[0,0]
    tn = tab.iloc[1,1]
    fp = tab.iloc[0,1]
    fn = tab.iloc[1,0]
    tot = tp+tn+fp+fn
    #
    # Compute performance statistics:
    acc = (tp+tn)/tot
    spec = tn/(tp+fn)
    sens = tp/(tp+fn)
    mcc = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    #
    # Return a dictionary including the confusion matrix and metrics:
    return({'confusion_matrix':tab,
            'accuracy':acc,
            'specificity':spec,
            'sensitivity':sens,
            'mcc':mcc})

print(performance(y_test, y_hat))

{'confusion_matrix': col_0    0    1
row_0          
0      878  140
1       11   11, 'accuracy': 0.8548076923076923, 'specificity': 0.012373453318335208, 'sensitivity': 0.9876265466816648, 'mcc': 0.14805547209346345}
